## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

import warnings
warnings.filterwarnings("ignore")

`(3) langfuase handler 설정`

In [3]:
from langfuse.langchain import CallbackHandler

# LangChain 콜백 핸들러 생성
langfuse_handler = CallbackHandler()

`(4) Langfuse 클라이언트 설정`

In [4]:
from langfuse import get_client

# Langfuse 클라이언트 초기화
langfuse = get_client()

# 연결 테스트
assert langfuse.auth_check()

---

## **프롬프트 관리**

- Langfuse는 **프롬프트 CMS** 기능 제공

- **버전 관리** 기능으로 롤백과 성능 비교 가능

### 1. **프롬프트 생성**

In [5]:
# 텍스트 프롬프트 생성
langfuse.create_prompt(
    name="movie-critic",  # 프롬프트 이름
    type="text",          
    prompt="{{criticLevel}} 영화 평론가로서, {{movie}}를 어떻게 생각하시나요?",
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa", "text"],    # 태그
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.7
    }
)

In [6]:
# 챗 프롬프트 생성
langfuse.create_prompt(
    name="movie-critic-chat",  # 프롬프트 이름
    type="chat",          
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다."
        },
        {
            "role": "user",
            "content": "영화 평론가로서, {{movie}}를 어떻게 생각하시나요?"
        }
    ], #type: ignore
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa", "chat"],    # 태그
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.7
    }
)  

### **[실습]**

- text 기반 프롬프트와 chat 기반 프롬프트를 각각 구현하고, Langfuse UI에서 확인하세요. 

In [ ]:
# 텍스트 프롬프트 생성

In [ ]:
# 챗 프롬프트 생성


### 2. **프롬프트 활용**

In [7]:
# 프로덕션 버전 가져오기
prompt = langfuse.get_prompt("movie-critic")

# 프롬프트 출력
print(f"모델: {prompt.config['model']}")
print(f"온도: {prompt.config['temperature']}")
print(f"라벨: {prompt.labels}")
print(f"태그: {prompt.tags}")
print(f"프롬프트: {prompt.prompt}")
print("-" * 100)

# 랭체인 프롬프트 출력
print(f"프롬프트: {prompt.get_langchain_prompt()}")

모델: gpt-4.1-mini
온도: 0.7
라벨: ['production', 'latest']
태그: ['movie', 'qa', 'text']
프롬프트: {{criticLevel}} 영화 평론가로서, {{movie}}를 어떻게 생각하시나요?
----------------------------------------------------------------------------------------------------
프롬프트: {criticLevel} 영화 평론가로서, {movie}를 어떻게 생각하시나요?


### **[실습]**

- "movie-critic-chat" 프롬프트를 Langfuse에서 가져와서 내용을 출력합니다.

In [ ]:
# chat 프롬프트 가져오기

In [8]:
# 프로덕션 버전 가져오기
prompt = langfuse.get_prompt("movie-critic-chat")

# 프롬프트 출력
print(f"모델: {prompt.config['model']}")
print(f"온도: {prompt.config['temperature']}")
print(f"라벨: {prompt.labels}")
print(f"태그: {prompt.tags}")
print(f"프롬프트: {prompt.prompt}")
print("-" * 100)

# 랭체인 프롬프트 출력
print(f"프롬프트: {prompt.get_langchain_prompt()}")

모델: gpt-4.1-mini
온도: 0.7
라벨: ['production', 'latest']
태그: ['movie', 'qa', 'chat']
프롬프트: [{'role': 'system', 'content': '당신은 {{criticLevel}} 영화 평론가입니다.'}, {'role': 'user', 'content': '영화 평론가로서, {{movie}}를 어떻게 생각하시나요?'}]
----------------------------------------------------------------------------------------------------
프롬프트: [('system', '당신은 {criticLevel} 영화 평론가입니다.'), ('user', '영화 평론가로서, {movie}를 어떻게 생각하시나요?')]


In [9]:
# 새로운 버전 생성
langfuse.create_prompt(
    name="movie-critic",  # 같은 이름 사용
    type="text",          
    prompt="당신은 {{criticLevel}} 영화 평론가입니다.\n\n영화 {{movie}}를 어떻게 생각하시나요?",
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa"],    # 태그
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.7
    }
)

In [14]:
# 특정 버전 가져오기
prompt_v1 = langfuse.get_prompt("movie-critic", version=1)

# 프롬프트 출력
print(f"모델: {prompt_v1.config['model']}")
print(f"온도: {prompt_v1.config['temperature']}")
print(f"라벨: {prompt_v1.labels}")
print(f"프롬프트: {prompt_v1.prompt}")

모델: gpt-4.1-mini
온도: 0.7
라벨: []
프롬프트: {{criticLevel}} 영화 평론가로서, {{movie}}를 어떻게 생각하시나요?


### **[실습]**

- "movie-critic-chat" 프롬프트를 수정하고, labels 속성은 "staging"으로 커스텀 지정하세요.
- Langfuse에서 "staging" 버전을 가져와서 내용을 출력합니다.

In [ ]:
# staging 버전 생성


In [ ]:
# staging 버전 가져오기

In [15]:
prompt_v1.get_langchain_prompt()

'{criticLevel} 영화 평론가로서, {movie}를 어떻게 생각하시나요?'

In [16]:
from langchain_core.prompts import PromptTemplate

# Langchain과 통합 - 'text' 프롬프트
langchain_prompt = PromptTemplate.from_template(
    prompt_v1.get_langchain_prompt(),
    metadata={"langfuse_prompt": prompt_v1},
)

print(langchain_prompt.format(criticLevel="비평가", movie="인셉션"))

비평가 영화 평론가로서, 인셉션를 어떻게 생각하시나요?


In [17]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화 (프롬프트 설정에서 가져온 값 사용)
model = ChatOpenAI(
    model=prompt.config.get("model", "gpt-4.1"),
    temperature=prompt.config.get("temperature", 0.1)
)

# 체인 생성 및 실행
chain = langchain_prompt | model
response = chain.invoke(
    input={"criticLevel": "전문가", "movie": "인셉션"},
    config={"callbacks": [langfuse_handler]}  # 콜백 핸들러 추가
)

# 응답 출력
print(response.content)

크리스토퍼 놀란 감독의 **《인셉션》(Inception, 2010)**은 현대 영화사에서 가장 독창적이고 사유를 자극하는 작품 중 하나로 평가받습니다. 영화 평론가로서 《인셉션》을 다음과 같은 측면에서 높이 평가합니다.

1. **독창적인 스토리텔링과 콘셉트**  
   《인셉션》은 꿈 속의 꿈, 다중 레이어의 의식 세계를 탐구하는 복잡하면서도 정교한 내러티브 구조를 선보입니다. 꿈을 통해 타인의 무의식을 조작한다는 독특한 설정은 기존의 SF나 스릴러 장르와 차별화되며, 관객으로 하여금 현실과 비현실의 경계를 끊임없이 고민하게 만듭니다.

2. **철학적이고 심리적인 깊이**  
   영화는 기억, 죄책감, 현실과 환상의 경계 등 인간 내면의 심리적 주제를 다룹니다. 특히 도미닉 코브(레오나르도 디카프리오 분)의 감정적 갈등과 성장 과정은 단순한 액션 블록버스터를 넘어 인간 존재에 대한 깊은 성찰을 가능하게 합니다.

3. **정교한 시각 효과와 연출**  
   《인셉션》은 꿈의 세계를 표현하기 위한 혁신적인 시각 효과와 미장센으로 찬사를 받았습니다. 공간이 왜곡되고 시간의 흐름이 다층적으로 변하는 장면들은 놀란 감독 특유의 치밀한 연출력과 함께 영화적 경험을 극대화합니다.

4. **음악과 편집의 완벽한 조화**  
   한스 짐머의 강렬하면서도 서정적인 사운드트랙은 영화의 긴장감과 몰입도를 한층 높입니다. 빠른 템포와 느린 템포가 교차하는 편집은 꿈 속 시간의 상대성을 효과적으로 전달하며, 관객을 끌어들이는 데 중요한 역할을 합니다.

5. **장르의 경계를 넘나드는 작품성**  
   액션, SF, 스릴러, 심리 드라마의 요소들을 결합하여 대중성과 예술성을 모두 충족시킨 점도 인상적입니다. 이는 《인셉션》이 단순한 상업 영화가 아니라 하나의 예술 작품으로 인정받는 이유 중 하나입니다.

종합적으로, 《인셉션》은 복잡한 구조와 철학적 질문을 대중적으로 풀어내는 데 성공한 걸작으로, 크리스토퍼 놀란 감독의 필모그래피에서 중요한 위치를 차지하며, 현대 영화의

### **[실습]**

- 앞에서 직접 정의한 text 기반 프롬프트를 가져와서, LangChain과 통합하여 Tracing 추적을 실행합니다. 
- 실행 결과를 Langfuse UI에서 확인하세요. 

In [ ]:
# 여기에 코드를 추가하세요.

In [18]:
# 특정 라벨 가져오기
prompt_staging = langfuse.get_prompt("movie-critic-chat", label="latest")  # production, latest

# 프롬프트 출력
print(f"모델: {prompt_staging.config['model']}")
print(f"온도: {prompt_staging.config['temperature']}")
print(f"라벨: {prompt_staging.labels}")
print(f"프롬프트: {prompt_staging.prompt}")

모델: gpt-4.1-mini
온도: 0.7
라벨: ['production', 'latest']
프롬프트: [{'role': 'system', 'content': '당신은 {{criticLevel}} 영화 평론가입니다.'}, {'role': 'user', 'content': '영화 평론가로서, {{movie}}를 어떻게 생각하시나요?'}]


In [20]:
from langchain_core.prompts import ChatPromptTemplate

# Langchain과 통합 - 'chat' 프롬프트
langchain_prompt = ChatPromptTemplate.from_messages(
    prompt_staging.get_langchain_prompt(type="chat"),
)
langchain_prompt.metadata = {"langfuse_prompt": prompt_staging}

print(langchain_prompt.format(criticLevel="비평가", movie="인셉션"))

System: 당신은 비평가 영화 평론가입니다.
Human: 영화 평론가로서, 인셉션를 어떻게 생각하시나요?


In [21]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화 (프롬프트 설정에서 가져온 값 사용)
model = ChatOpenAI(
    model=prompt.config.get("model", "gpt-4.1-mini"),
    temperature=prompt.config.get("temperature", 0.7)
)

# 체인 생성 및 실행
chain = langchain_prompt | model
response = chain.invoke(
    input={"criticLevel": "전문가", "movie": "인셉션"},
    config={"callbacks": [langfuse_handler]}  # 콜백 핸들러 추가
)

# 응답 출력
print(response.content)

크리스토퍼 놀란 감독의 2010년작 『인셉션(Inception)』은 현대 SF 스릴러 장르에서 독보적인 위치를 차지하는 작품입니다. 이 영화는 꿈과 현실의 경계를 탐구하며, 복잡한 내러티브 구조와 시각적 효과를 통해 관객에게 깊은 몰입감을 선사합니다.

첫째, 각본과 플롯의 치밀함이 돋보입니다. 여러 층위의 꿈속을 넘나드는 이야기 전개는 관객으로 하여금 끊임없이 현실과 환상의 경계를 의심하게 만들며, 이는 영화의 주제인 '현실 인식'에 대한 철학적 질문과 맞닿아 있습니다.

둘째, 배우들의 연기와 캐릭터 구성도 탁월합니다. 레오나르도 디카프리오가 연기한 도미닉 코브는 복잡한 내면 갈등과 상실감을 섬세하게 표현하며, 각 인물들은 꿈의 세계에서 각자의 역할을 충실히 수행합니다.

셋째, 한스 짐머의 음악과 함께하는 시각적 효과는 영화의 긴장감과 몰입도를 극대화합니다. 특히 무중력 장면과 도시가 접히는 시퀀스는 혁신적인 CG 기술과 실사 촬영의 조화로 인해 아직도 많은 이들의 기억에 남아 있습니다.

마지막으로, 『인셉션』은 단순한 오락 영화 이상의 의미를 지니며, 인간의 무의식과 기억, 그리고 현실 인식에 대한 깊은 사유를 이끌어냅니다. 이러한 점에서 이 작품은 영화 예술과 대중문화 모두에 큰 영향을 끼친 명작으로 평가받기에 충분합니다.

요약하자면, 『인셉션』은 탁월한 연출과 서사, 그리고 철학적 깊이를 겸비한 작품으로, 영화 평론가로서 매우 높은 평가를 내릴 수밖에 없는 영화입니다.


### **[실습]**

- 앞에서 직접 정의한 chat 기반 프롬프트를 가져와서, LangChain과 통합하여 Tracing 추적을 실행합니다. 
- 실행 결과를 Langfuse UI에서 확인하세요. 

In [ ]:
# 여기에 코드를 추가하세요.